In [12]:
import os
import pandas as pd

In [1]:
import sys
sys.path.append('../code')

In [2]:
%load_ext autoreload
%autoreload 2

from libs import constants
from libs import io
from libs import scholar

In [3]:
aps_os_data_tar_gz = '../data/final_dataset.tar.gz'
aps_data_path = '../data/aps_20240130'

In [4]:
# 1. Read aps-oa data
df_authors = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_AUTHORS_FN)
print(f'\n df_authors: {df_authors.shape}  \n {df_authors.head(5)} \n')


 df_authors: (481012, 10)  
     id_author created_date                updated_date          name  \
0  5053051063   2023-07-21  2024-11-04T00:46:12.391095    I. Ben‐Zvi   
1  5067224934   2023-07-21  2024-11-06T10:20:37.304362      T. Roser   
2  5012146130   2023-07-21  2024-11-01T15:52:34.664125   M. Ferrario   
3  5051894783   2023-07-21  2024-11-01T03:49:19.361803  Jean-Luc Vay   
4  5041648606   2023-07-21  2024-11-02T20:12:56.799035   A. Friedman   

                 orcid  two_year_mean_citedness  h_index  i10_index  \
0  0000-0001-5583-0106                 5.416667       33        129   
1  0000-0001-5603-3192                 0.322581       34         89   
2  0000-0002-1105-0359                 2.535714       45        146   
3  0000-0002-0040-799X                 1.581395       34        101   
4  0000-0003-0421-2476                 0.428571       28         97   

   works_count  cited_by_count  
0          783            6034  
1          632            5487  
2          

In [5]:
# 2. authorships
df_authorships = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_AUTHORSHIPS_FN)
print(f'\n df_authorships: {df_authorships.shape}  \n {df_authorships.head(5)} \n')


 df_authorships: (3111978, 5)  
    id_publication   id_author position  id_institution    authorship_flag
0      2155512932  5061426463    first    4.210121e+09  original_openalex
1      2155512932  5064510725   middle    1.282106e+09  original_openalex
2      2155512932  5111938223   middle    4.210121e+09  original_openalex
3      2155512932  5052336272   middle    4.210121e+09  original_openalex
4      2155512932  5110594767   middle    4.210121e+09  original_openalex 



In [6]:
# 3. publications
df_publications = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_PUBLICATIONS_FN)
print(f'\n df_publications: {df_publications.shape}  \n {df_publications.head(5)} \n')


 df_publications: (678874, 18)  
    id_publication created_date                updated_date  \
0      2155512932   2016-06-24  2024-10-20T07:51:36.512074   
1      1990233377   2016-06-24  2024-11-02T19:41:40.382208   
2      2006493953   2016-06-24  2024-11-02T02:20:53.552587   
3      2035445126   2016-06-24  2024-10-07T20:59:41.180954   
4      1976015478   2016-06-24  2024-10-19T02:58:20.325244   

                                               title  \
0  Suppression of microbunching instability in th...   
1              Cryogenic permanent magnet undulators   
2  Induction voltage adders and the induction acc...   
3  Review of single bunch instabilities driven by...   
4  Three-dimensional time and frequency-domain th...   

                            doi  cited_by_count  has_full_text  \
0  10.1103/physrevstab.7.074401             221          False   
1  10.1103/physrevstab.7.050702             143          False   
2  10.1103/physrevstab.7.064801             108          

In [7]:
# 4. citations
df_citations = io.read_file_from_tar_gz_as_dataframe(aps_os_data_tar_gz, constants.APS_OA_CITATIONS_FN)
print(f'\n df_citations: {df_citations.shape}  \n {df_citations.head(5)} \n')


 df_citations: (19501208, 2)  
    id_publication_citing  id_publication_cited
0             2155512932            1588822808
1             2155512932            1981960059
2             2155512932            2005431074
3             2155512932            2010040766
4             2155512932            2032874831 



In [28]:
df_citations_author = df_citations.merge(df_authorships[['id_publication','id_author']], left_on='id_publication_cited', right_on='id_publication', how='left').drop(columns=['id_publication'])
df_citations_author_paper = df_citations_author.merge(df_publications[['id_publication','publication_date']], left_on='id_publication_cited', right_on='id_publication', how='left')
df_citations_author.shape, df_citations_author_paper.shape

((55220132, 3), (55220132, 5))

In [29]:
df_citations_author_paper.query("id_publication_cited==2155512932 and ~@pd.isna(id_publication)")

,id_publication_citing,id_publication_cited,id_author,id_publication,publication_date
434,2141009273,2155512932,5.061426e+09,2.155513e+09,2004-07-08
435,2141009273,2155512932,5.064511e+09,2.155513e+09,2004-07-08
436,2141009273,2155512932,5.111938e+09,2.155513e+09,2004-07-08
437,2141009273,2155512932,5.052336e+09,2.155513e+09,2004-07-08
438,2141009273,2155512932,5.110595e+09,2.155513e+09,2004-07-08
...,...,...,...,...,...
54752961,2346228725,2155512932,5.111938e+09,2.155513e+09,2004-07-08
54752962,2346228725,2155512932,5.052336e+09,2.155513e+09,2004-07-08
54752963,2346228725,2155512932,5.110595e+09,2.155513e+09,2004-07-08
54752964,2346228725,2155512932,5.050315e+09,2.155513e+09,2004-07-08


In [30]:
df_e_index = df_citations_author_paper.groupby('id_author').apply(lambda group: scholar.compute_e_index(group, col_publication='id_publication_cited', col_citation_from='id_publication_citing')).reset_index(name='e_index')
df_e_index.shape, df_authors.shape
    

/tmp/ipykernel_3003427/2099771938.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_e_index = df_citations_author_paper.groupby('id_author').apply(lambda group: scholar.compute_e_index(group, col_publication='id_publication_cited', col_citation_from='id_publication_citing')).reset_index(name='e_index')


((421453, 2), (481012, 10))

In [35]:
df_e_index.loc[236191].astype(int)

id_author    5068441007
e_index              27
Name: 236191, dtype: int64

In [31]:
df_authorships_paper = df_authorships.merge(df_publications[['id_publication','publication_date']], on='id_publication', how='left')
df_authorships_paper['year'] = df_authorships_paper['publication_date'].str.split('-').str[0].astype(int)
df_authorships_paper.head(2)

,id_publication,id_author,position,id_institution,authorship_flag,publication_date,year
0,2155512932,5061426463,first,4.210121e+09,original_openalex,2004-07-08,2004
1,2155512932,5064510725,middle,1.282106e+09,original_openalex,2004-07-08,2004


In [32]:
df_time = df_authorships_paper.groupby('id_author').year.agg(['min','max']).reset_index().rename(columns={'min':'min_year','max':'max_year'})
df_time['career_age'] = df_time.max_year - df_time.min_year
df_time.head(2)

,id_author,min_year,max_year,career_age
0,5000000199,2016,2018,2
1,5000000755,1994,1994,0
